# Organize freezed data
Aug 19th, 2019

Sep 13th, 2019 (data freeze)

Fangming Xie

- reformat raw data (from the dropbox) into standardized python readable format
- use ensembl id to refer genes
- preserve content as much as possible, including naming conventions of table header and index

In [2]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_clst_utils
from scipy.io import mmread
from scipy import sparse
import time


In [3]:
SRC_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze'
DST_DIR = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted'

In [4]:
f = PATH_GENEBODY_ANNOTATION 
df_genes = pd.read_csv(f, sep="\t")
df_genes['gene_id'] = df_genes['gene_id'].apply(lambda x: x.split('.')[0])
df_genes = df_genes.set_index('gene_id')
df_genes_v2 = df_genes.reset_index().groupby('gene_name').first()
print(df_genes.shape)
df_genes.head()
print(df_genes_v2.shape)
df_genes_v2.head()

(53379, 6)
(53278, 6)


,gene_id,chr,start,end,strand,gene_type
gene_name,,,,,,
0610005C13Rik,ENSMUSG00000109644,chr7,45567794,45575327,-,antisense_RNA
0610006L08Rik,ENSMUSG00000108652,chr7,74818818,74853813,-,lincRNA
0610009B22Rik,ENSMUSG00000007777,chr11,51685386,51688874,-,protein_coding
0610009E02Rik,ENSMUSG00000086714,chr2,26445696,26459390,+,processed_transcript
0610009L18Rik,ENSMUSG00000043644,chr11,120348678,120351190,+,bidirectional_promoter_lncRNA


### 10x_nuclei_v3_macosko 
- 2 metadata .csv
- 1 data matrix in .rds format
- concatenate male and female, get autosomo genes only

In [8]:
dataset_name = '10x_nuclei_v3_macosko'


# outdated
f_meta = os.path.join(SRC_DIR, dataset_name, 'motor_cortex_v3_mop_data_metadata.csv') 
# f_meta4 = os.path.join(SRC_DIR, dataset_name, 'zizhen_processing', 'sample_metadata.csv') 

# zizhen's list 
f_meta1 = os.path.join(SRC_DIR, dataset_name, 'zizhen_processing', 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'zizhen_processing', 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'zizhen_processing', 'QC.csv') 

f_data = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_female.mtx') 
f_data_gene = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_female_gene.csv') 
f_data_cell = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_female_cell.csv') 

f_data2 = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_male.mtx') 
f_data2_gene = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_male_gene.csv') 
f_data2_cell = os.path.join(SRC_DIR, dataset_name, './reformatted/raw_male_cell.csv') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

# print(f_meta)
print(fout_meta)
print(f_data)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_nuclei_v3_macosko_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_nuclei_v3_macosko/./reformatted/raw_female.mtx
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_nuclei_v3_macosko_counts.npz


In [5]:
ti = time.time()
data = mmread(f_data)
gene = pd.read_csv(f_data_gene)['x'].values
cell = pd.read_csv(f_data_cell)['x'].values
print(time.time()-ti)

data2 = mmread(f_data2)
gene2 = pd.read_csv(f_data2_gene)['x'].values
cell2 = pd.read_csv(f_data2_cell)['x'].values
print(time.time()-ti)

672.5768630504608
1225.8807742595673


In [6]:
# assert np.all(gene==gene2)
# gene.shape
# gene2.shape

In [7]:
# autosome genes
chromosomes = ['chr{}'.format(chrom) for chrom in snmcseq_utils.get_mouse_chromosomes(include_x=False)]
gene_table1 = df_genes_v2[df_genes_v2['chr'].isin(chromosomes)].reindex(gene).dropna()
gene_table2 = df_genes_v2[df_genes_v2['chr'].isin(chromosomes)].reindex(gene2).dropna()

genes_common = np.intersect1d(gene_table1.index.values, gene_table2.index.values)
genes_common_ids = gene_table1.loc[genes_common, 'gene_id'].values 
print(gene.shape, 
      gene2.shape, 
      genes_common.shape, 
      genes_common_ids.shape, 
     )

selected_genes_idx1 = snmcseq_utils.get_index_from_array(gene, genes_common)
selected_genes_idx2 = snmcseq_utils.get_index_from_array(gene2, genes_common)
print(selected_genes_idx1.shape, 
      selected_genes_idx2.shape, 
     )

# 
data_comb = sparse.hstack([
                data.tocsr()[selected_genes_idx1,:], data2.tocsr()[selected_genes_idx2,:]
            ])
print(data_comb.shape)

(27574,) (27495,) (24809,) (24809,)
(24809,) (24809,)
(24809, 215823)


In [8]:
# output
# save 
ti = time.time()
gc_mat = GC_matrix(genes_common_ids, 
                   np.hstack([cell, cell2]), 
                   data_comb, 
                  )
snmcseq_utils.save_gc_matrix(gc_mat, fout_data_gene, fout_data_cell, fout_data)
print(time.time()-ti)

990.3496780395508


In [12]:
# # assemble metadata (keep order, add male and female)

meta = pd.read_csv(f_meta, index_col=0).rename(columns={'dataset': 'sex'})[['nUMI', 'nGene', 'sex']]
# meta.index.name = 'cell'
# meta = meta.loc[np.hstack([cell, cell2])]
print(meta.shape)
meta.head()

# # meta.to_csv(fout_meta, sep="\t", header=True, index=True)

(215823, 3)


,nUMI,nGene,sex
pBICCNsMMrMOpRMiM003d190318_AAACCCACACAAGCTT,4859,2336,MALE
pBICCNsMMrMOpRMiM003d190318_AAACCCACAGTCCCGA,24082,6164,MALE
pBICCNsMMrMOpRMiM003d190318_AAACCCAGTGAATTAG,5194,2519,MALE
pBICCNsMMrMOpRMiM003d190318_AAACCCAGTGGTAACG,1343,998,MALE
pBICCNsMMrMOpRMiM003d190318_AAACCCAGTTCCAAAC,9022,3558,MALE


In [6]:

meta1 = pd.read_csv(f_meta1, names=['cell', 'cluster_id'], header=0) # cluster assignment
meta2 = pd.read_csv(f_meta2) # cluster annot [all metrics]
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0) # QC'ed cells
meta1 = pd.merge(meta1, meta3, on='cell') # cluster id on QC'ed cells
meta1 = pd.merge(meta1, meta2, on='cluster_id')
meta1 = pd.merge(meta1, meta, left_on='cell', right_index=True)

print(meta1.shape)
meta1.head()

(159738, 16)


,cell,cluster_id,QC,cluster_label,cluster_color,class_label,subclass_label,size,gene.counts,umi.counts,Broad.QC.doublet,Broad.QC.Mito,Broad.passQC,MALE,Comb.QC,cl
0,pBICCNsMMrMOpRMiM003d190318_AAACCCACACAAGCTT,90,1,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150
1,pBICCNsMMrMOpRMiM003d190318_AAAGGATCAGACCGCT,90,19,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150
2,pBICCNsMMrMOpRMiM003d190318_AAAGGATCATGGAACG,90,20,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150
3,pBICCNsMMrMOpRMiM003d190318_AAAGGTAGTAATACCC,90,37,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150
4,pBICCNsMMrMOpRMiM003d190318_AAAGTCCCAAGGTCAG,90,39,Oligo Opalin_4,#474662,Non-Neuronal,Oligo,16566,1740.348364,3419.937704,0.06,0.0,0.94,0.487323,NaN,150


In [29]:
meta1.to_csv(fout_meta, sep="\t", header=True, index=False)

### 10x_nuclei_v3

- gene_ids
- all cells passed QC (QC.csv) metadata

- all cells data
- output

In [73]:
dataset_name = '10x_nuclei_v3'

f_meta = os.path.join(SRC_DIR, dataset_name, 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'QC.csv') 

f_data = os.path.join(SRC_DIR, dataset_name, 'matrix.mtx.gz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name, 'features.tsv.gz') 
f_data_cell = os.path.join(SRC_DIR, dataset_name, 'barcodes.tsv.gz') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

print(f_meta)
print(fout_meta)
print(f_data)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_nuclei_v3/cluster.membership.csv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_nuclei_v3_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_nuclei_v3/matrix.mtx.gz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_nuclei_v3_counts.npz


In [74]:
meta = pd.read_csv(f_meta, names=['cell', 'cluster_id'], header=0) 
meta2 = pd.read_csv(f_meta2)
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0)
meta = pd.merge(meta, meta3, on='cell')
meta = pd.merge(meta, meta2, on='cluster_id')

print(meta.shape)
meta.head()

(40166, 8)


,cell,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
0,AAACCCAAGCTCTTCC-1L8TX_181211_01_A02,59,1,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
1,AAAGGGCTCGCGATCG-1L8TX_181211_01_A02,59,47,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
2,AAAGTGACATCGCTGG-1L8TX_181211_01_A02,59,80,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
3,AACAACCGTGCTCTCT-1L8TX_181211_01_A02,59,108,L6b Shisa6,L6b,Glutamatergic,#2B9880,247
4,AACAAGAAGCCTGCCA-1L8TX_181211_01_A02,59,112,L6b Shisa6,L6b,Glutamatergic,#2B9880,247


In [75]:
data_cell = snmcseq_utils.import_single_textcol(f_data_cell)
data_gene = snmcseq_utils.import_single_textcol(f_data_gene, col=0)
data = mmread(f_data)
print(data_cell.shape)
print(data_gene.shape)
print(data.shape)
data_cell
data_gene

(90266,)
(31053,)
(31053, 90266)


array(['ENSMUSG00000051951', 'ENSMUSG00000089699', 'ENSMUSG00000102343',
       ..., 'ENSMUSG00000063897', 'ENSMUSG00000096730',
       'ENSMUSG00000095742'], dtype=object)

In [79]:
gc_mat = GC_matrix(data_gene, data_cell, data)

In [80]:
meta.to_csv(fout_meta, sep="\t", header=True, index=False)

In [81]:
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

### 10x_cells_v3 

In [84]:
dataset_name = '10x_cells_v3'

f_meta = os.path.join(SRC_DIR, dataset_name, 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'QC.csv') 

f_data = os.path.join(SRC_DIR, dataset_name, 'matrix.mtx.gz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name, 'features.tsv.gz') 
f_data_cell = os.path.join(SRC_DIR, dataset_name, 'barcodes.tsv.gz') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

print(f_meta)
print(fout_meta)
print(f_data)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_cells_v3/cluster.membership.csv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_cells_v3_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_cells_v3/matrix.mtx.gz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_cells_v3_counts.npz


In [85]:
meta = pd.read_csv(f_meta, names=['cell', 'cluster_id'], header=0) 
meta2 = pd.read_csv(f_meta2)
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0)
meta = pd.merge(meta, meta3, on='cell')
meta = pd.merge(meta, meta2, on='cluster_id')

print(meta.shape)
meta.head()

(71183, 8)


,cell,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
0,AAACCCAAGCTTCATG-1L8TX_181211_01_G12,42,1,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
1,AAACCCACACCAGCCA-1L8TX_181211_01_G12,42,3,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
2,AAACGAACAACGATTC-1L8TX_181211_01_G12,42,8,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
3,AAACGAATCTCGTGAA-1L8TX_181211_01_G12,42,11,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334
4,AAACGCTGTAGTCACT-1L8TX_181211_01_G12,42,14,L5 IT Tcap_2,L5 IT,Glutamatergic,#52CA74,17334


In [86]:
data_cell = snmcseq_utils.import_single_textcol(f_data_cell)
data_gene = snmcseq_utils.import_single_textcol(f_data_gene, col=0)
data = mmread(f_data)
print(data_cell.shape)
print(data_gene.shape)
print(data.shape)
data_cell
data_gene

gc_mat = GC_matrix(data_gene, data_cell, data)

(176584,)
(31053,)
(31053, 176584)


In [87]:
meta.to_csv(fout_meta, sep="\t", header=True, index=False)

In [88]:
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

### 10x_cells_v2 

In [89]:
dataset_name = '10x_cells_v2'

f_meta = os.path.join(SRC_DIR, dataset_name, 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'QC.csv') 

f_data = os.path.join(SRC_DIR, dataset_name, 'matrix.mtx.gz') 
f_data_gene = os.path.join(SRC_DIR, dataset_name, 'features.tsv.gz') 
f_data_cell = os.path.join(SRC_DIR, dataset_name, 'barcodes.tsv.gz') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

print(f_meta)
print(fout_meta)
print(f_data)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_cells_v2/cluster.membership.csv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_cells_v2_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze/10x_cells_v2/matrix.mtx.gz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/10x_cells_v2_counts.npz


In [90]:
meta = pd.read_csv(f_meta, names=['cell', 'cluster_id'], header=0) 
meta2 = pd.read_csv(f_meta2)
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0)
meta = pd.merge(meta, meta3, on='cell')
meta = pd.merge(meta, meta2, on='cluster_id')

print(meta.shape)
meta.head()

(122641, 8)


,cell,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size
0,AAACCTGAGGAGTCTG-1L8TX_171026_01_F03,9,1,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
1,ACCTTTAGTACAGCAG-1L8TX_171026_01_F03,9,673,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
2,ACGATACCACCCAGTG-1L8TX_171026_01_F03,9,724,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
3,ACGGGTCAGTGGGCTA-1L8TX_171026_01_F03,9,885,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293
4,AGCAGCCCAGTTCATG-1L8TX_171026_01_F03,9,1205,Sncg Slc17a8,Sncg,GABAergic,#9440F3,293


In [91]:
data_cell = snmcseq_utils.import_single_textcol(f_data_cell)
data_gene = snmcseq_utils.import_single_textcol(f_data_gene, col=0)
data = mmread(f_data)
print(data_cell.shape)
print(data_gene.shape)
print(data.shape)
data_cell
data_gene

gc_mat = GC_matrix(data_gene, data_cell, data)

(145748,)
(31053,)
(31053, 145748)


In [92]:
meta.to_csv(fout_meta, sep="\t", header=True, index=False)

In [93]:
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

### Smart-cells

In [8]:
dataset_name = 'smarter_cells'

f_meta = os.path.join(SRC_DIR, dataset_name, 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'QC.csv') 
f_meta4 = os.path.join(SRC_DIR, dataset_name, 'sample_metadata.csv.gz') 

f_data_intron = os.path.join(SRC_DIR, dataset_name, 'intron.counts.csv.gz') 
f_data_exon = os.path.join(SRC_DIR, dataset_name, 'exon.counts.csv.gz') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

print(f_meta)
print(fout_meta)
# print(f_data)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze/smarter_cells/cluster.membership.csv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/smarter_cells_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/smarter_cells_raw.npz


In [6]:
meta = pd.read_csv(f_meta, names=['cell', 'cluster_id'], header=0) 
meta2 = pd.read_csv(f_meta2)
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0)
meta4 = pd.read_csv(f_meta4, index_col=0)
meta4.index.name = 'cell' 
meta4 = meta4.reset_index()

meta = pd.merge(meta, meta3, on='cell')
meta = pd.merge(meta, meta2, on='cluster_id')
meta = pd.merge(meta, meta4, on='cell')

print(meta.shape)
print(meta3.shape)
meta.head()

(6288, 129)
(6288, 2)


,cell,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size,ar_id,exp_component_vendor_name,...,Injection_type,NewBatch,Vendor,MultiPlex,percent_reads_aligned_to_rrna_rmsk,percent_reads_aligned_to_trna_rmsk,percent_reads_aligned_to_ncrna,percent_reads_aligned_to_introns,percent_reads_aligned_to_intergenic,percent_reads_aligned_unique
0,LS-15395_S41_E1-50,19,1,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,543183476,LS-15395-41,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.162980,9.855384,3.264771,84.993663
1,LS-15395_S42_E1-50,19,2,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,543183474,LS-15395-42,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.281620,6.763949,2.856108,87.252036
2,LS-15501_S28_E1-50,19,94,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,544930823,LS-15501-28,...,0.0,HistoricalData,Covance,index96,NaN,NaN,17.066841,3.311346,2.437971,86.831969
3,LS-15501_S81_E1-50,19,120,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,544930707,LS-15501-81,...,0.0,HistoricalData,Covance,index96,NaN,NaN,13.866006,11.769028,3.699682,85.252590
4,LS-15524_S21_E1-50,19,267,Sst Myh8_1,Sst,GABAergic,#FF7F2C,136,549999237,LS-15524-21,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.912637,5.923318,2.385434,82.503705


In [9]:
data_intron = pd.read_csv(f_data_intron, index_col=0) 
data_exon = pd.read_csv(f_data_exon, index_col=0) 

data = data_intron + data_exon
gene = data.index.values
cell = data.columns.values

gene_table = df_genes_v2.reindex(gene).dropna()
gene_ids = gene_table['gene_id'].values
gene_names = gene_table.index.values

print(gene_ids.shape, gene_names.shape)
print(data.shape)

(32324,) (32324,)
(45768, 6300)


In [10]:
data_sparse = sparse.coo_matrix(data.loc[gene_names])
gene = gene_ids

In [11]:
gc_mat = GC_matrix(gene, cell, data_sparse)

In [12]:
meta.to_csv(fout_meta, sep="\t", header=True, index=False)
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

### Smarter-nuclei

In [4]:
dataset_name = 'smarter_nuclei'

f_meta = os.path.join(SRC_DIR, dataset_name, 'cluster.membership.csv') 
f_meta2 = os.path.join(SRC_DIR, dataset_name, 'cluster.annotation.csv') 
f_meta3 = os.path.join(SRC_DIR, dataset_name, 'QC.csv') 
f_meta4 = os.path.join(SRC_DIR, dataset_name, 'sample_metadata.csv.gz') 

f_data_intron = os.path.join(SRC_DIR, dataset_name, 'intron.counts.csv.gz') 
f_data_exon = os.path.join(SRC_DIR, dataset_name, 'exon.counts.csv.gz') 

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

print(f_meta)
print(fout_meta)
print(fout_data)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze/smarter_nuclei/cluster.membership.csv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/smarter_nuclei_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/smarter_nuclei_raw.npz


In [11]:
meta = pd.read_csv(f_meta, names=['cell', 'cluster_id'], header=0) 
meta2 = pd.read_csv(f_meta2)
meta3 = pd.read_csv(f_meta3, names=['QC', 'cell'], header=0)

meta4 = pd.read_csv(f_meta4, index_col=0)
meta4.index.name = 'cell' 
meta4 = meta4.reset_index()

meta = pd.merge(meta, meta3, on='cell')
meta = pd.merge(meta, meta2, on='cluster_id')
meta = pd.merge(meta, meta4.drop(columns=['cluster_id', 'cluster_label', 'cluster_color', 'size']), 
                on='cell')

print(meta.shape)
print(meta3.shape)
# meta.filter(regex='_x$', axis=1).head()
meta.head()

(6171, 146)
(6171, 2)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cell,cluster_id,QC,cluster_label,subclass_label,class_label,cluster_color,size,ar_id,exp_component_vendor_name,...,pred.cl.2,pred.score.2,ref.cl.2,ref.cl.1.1,pred.cl.3,pred.score.3,cluster_label.2,category_label,Region.1,gene.counts
0,SM-GE653_S113_E1-50,2,1,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869537,SM-GE653-S113,...,14,0.917036,10,31,14,0.917036,Vip Htr1f,GABAergic,Cortex,7792
1,SM-GE653_S116_E1-50,2,4,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869531,SM-GE653-S116,...,3,0.712671,2,7,3,0.712671,Lamp5 Pdlim5,GABAergic,Cortex,8502
2,SM-GE653_S117_E1-50,2,5,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869529,SM-GE653-S117,...,12,0.899050,9,28,12,0.899050,Vip Chat_3,GABAergic,Cortex,7021
3,SM-GE653_S120_E1-50,2,8,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869523,SM-GE653-S120,...,11,0.892925,9,28,11,0.892925,Vip Chat_2,GABAergic,Cortex,7085
4,SM-GE653_S122_E1-50,2,10,Lamp5 Egln3_2,Lamp5,GABAergic,#E69B73,87,647869519,SM-GE653-S122,...,13,0.822251,9,28,13,0.822251,Vip Lmo1,GABAergic,Cortex,7516


In [35]:
data_intron = pd.read_csv(f_data_intron, index_col=0) 
data_exon = pd.read_csv(f_data_exon, index_col=0) 

data = data_intron + data_exon
cell = data.columns.values
gene = data.index.values

gene_table = df_genes_v2.reindex(gene).dropna()
gene_ids = gene_table['gene_id'].values
gene_names = gene_table.index.values

print(gene_ids.shape, gene_names.shape)
print(data.shape)

(32324,) (32324,)
(45768, 6278)


In [36]:
data_sparse = sparse.coo_matrix(data.loc[gene_names])

In [37]:
gc_mat = GC_matrix(gene_ids, cell, data_sparse)

In [12]:
meta.to_csv(fout_meta, sep="\t", header=True, index=False)
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

### mC

- QC'ed cells
- remove version on ensembl id

In [12]:
import xarray as xr

In [16]:
dataset_name = 'snmcseq_gene'

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data_mc = os.path.join(DST_DIR, dataset_name+'_mCH_counts.npz') 
fout_data_c = os.path.join(DST_DIR, dataset_name+'_CH_counts.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_counts.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_counts.cell') 

print(fout_meta)
print(fout_data_mc)
print(fout_data_c)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_mCH_counts.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snmcseq_gene_CH_counts.npz


In [17]:
# metadata after QC and clustering
# meta = pd.read_msgpack(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/Metadata.after_QC.msg'))
# meta2 = pd.read_msgpack(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/mop_mc_clustering/MOp.methylome_only.analysis_results.msg')).drop(columns=['cell'])
# print(meta.shape)
# print(meta2.shape)
# print(meta.columns.values.shape)
# print(np.intersect1d(meta.columns.values, meta2.columns.values).shape)
# meta2 = meta2.drop(columns=meta2.filter(regex='^ENSMUSG', axis=1).columns).drop(columns='AllcPath')

meta3 = pd.read_csv(os.path.join(SRC_DIR, 'snmc_and_snatac/study/mc/MOp.snmC-seq.AnalysisResult.csv.gz')).rename(columns={'index': 'cell'})
print(meta3.shape)
meta3.head()

(9876, 32)


,cell,FullCellID,CCC_Rate,CG_Rate,CG_RateAdj,CH_Rate,CH_RateAdj,FinalReads,InputReads,MappedReads,...,Slice,PassFilter,pca_0,pca_1,umap_0,umap_1,tsne_0,tsne_1,MajorCluster,SubCluster
0,2C_M_0,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00511,0.77073,0.76955,0.02154,0.01651,1076259.0,2314862,1517338.0,...,2,True,-11.588627,-0.005045,10.051389,7.099836,-3.343437,18.330174,L4-IT-Rorb,L4-IT-Rorb_Rorb-Tenm2
1,2C_M_1,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00531,0.75683,0.75553,0.02196,0.01674,1197092.0,2590626,1709334.0,...,2,True,-19.267288,-7.374098,13.717241,-1.786627,9.212860,-26.514785,L23-IT-Cux2,L23-IT-Cux2
2,2C_M_10,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00581,0.75720,0.75578,0.02603,0.02034,1006630.0,2348484,1606410.0,...,2,True,-15.340144,-2.739380,4.829485,9.943986,-22.960995,13.509483,Outlier,Outlier
3,2C_M_100,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...,0.00634,0.74984,0.74824,0.02718,0.02097,2026078.0,4599922,3154619.0,...,2,True,-23.625140,-5.929033,13.739521,1.107174,12.798545,-17.126331,L23-IT-Cux2,L23-IT-Cux2
4,2C_M_1000,180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...,0.00521,0.77073,0.76953,0.02117,0.01604,2368464.0,5320502,3648786.0,...,2,True,-21.815190,-7.130878,14.154541,-2.284118,8.443658,-27.943267,L23-IT-Cux2,L23-IT-Cux2


In [18]:
meta3.to_csv(fout_meta, sep="\t", header=True, index=False)

In [11]:
reps = ['2C-180409.mcds', 
        '2C-180410.mcds',
        '3C-171206.mcds',
        '3C-171207.mcds',
        '4B-171212.mcds',
        '4B-171213.mcds',
        '4B-180104.mcds',
        '5D-180605.mcds',
        '5D-180612.mcds',
       ]

c_tables = []
mc_tables = []
for rep in reps:
    mcds = xr.open_dataset(os.path.join(SRC_DIR, 'snmc_and_snatac/dataset/mc/{}'.format(rep)))
    c_table = mcds['gene_da'].sel(mc_type='CHN', count_type='cov').squeeze().to_pandas()
    mc_table = mcds['gene_da'].sel(mc_type='CHN', count_type='mc').squeeze().to_pandas()
    c_tables.append(c_table)
    mc_tables.append(mc_table)
    
c_tables = pd.concat(c_tables, axis=0)
mc_tables = pd.concat(mc_tables, axis=0)
assert np.all(c_tables.columns.values == mc_tables.columns.values)
assert np.all(c_tables.index.values == mc_tables.columns.values)
print(c_tables.shape, mc_tables.shape)
    
gene = c_tables.columns.values
cell = c_tables.index.values 
c_table = c_tables.T
c_table.index = gene

mc_table = mc_tables.T
mc_table.index = gene

In [26]:
# remove version on ensembl id
c_table = sparse.coo_matrix(c_table.values)
mc_table = sparse.coo_matrix(mc_table.values)

In [31]:
cell = c_tables.index.values
cell

array(['2C_M_10', '2C_M_0', '2C_M_1', ..., '5D_M_2370', '5D_M_2348',
       '5D_M_2367'], dtype=object)

In [34]:
# save mc and c tables

data = {'mc': mc_table, 
        'c': c_table, 
       }
gc_mat = GC_matrix(gene, cell, data) 

snmcseq_utils.save_gc_matrix_methylation(gc_mat, 
                                         fout_data_gene, fout_data_cell, 
                                         fout_data_mc, fout_data_c)

### ATAC 
- rename barcode and header
- read snap files

In [19]:
dataset_name = 'snatac_gene'

fout_meta = os.path.join(DST_DIR, dataset_name+'_metadata.tsv') 
fout_data = os.path.join(DST_DIR, dataset_name+'_raw.npz') 
fout_data_gene = os.path.join(DST_DIR, dataset_name+'_raw.gene') 
fout_data_cell = os.path.join(DST_DIR, dataset_name+'_raw.cell') 

print(fout_meta)
print(fout_data)
print(fout_data_gene)
print(fout_data_cell)

/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snatac_gene_metadata.tsv
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snatac_gene_raw.npz
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snatac_gene_raw.gene
/cndd/fangming/CEMBA/data/MOp_all/data_freeze_formatted/snatac_gene_raw.cell


In [20]:
f = ('/cndd/Public_Datasets/CEMBA/BICCN_minibrain_data/data_freeze/snmc_and_snatac/study/atac/'
     'yang_analysis_20190912/CEMBA_MOp.L2.cluster.meta.txt'
    )
meta_new = pd.read_csv(f, sep='\t').rename(columns={'x.sp.joint@sample': 'sample', 
                                                    'x.sp.joint@cluster': 'cluster', 
                                                   })
meta_cols = meta_new.columns.values
meta_new['cell'] = meta_new['sample'] + '_' + meta_new['barcode']
meta_new = meta_new[np.hstack([['cell'], meta_cols])]
print(meta_new.groupby('cluster').size())
print(meta_new.shape)
meta_new.head()

cluster
ASC                7215
Chodl                86
Endo               1245
L23.a              6085
L23.b              5758
L23.c              3590
L4                 7307
L5.IT.a            3668
L5.IT.b            4214
L5.PT              1755
L6.CT              8008
L6.IT              4233
Lamp5_Arhgdib       237
Lamp5_Mettl21e      171
Lamp5_Ndnf          204
Lamp5_Smad3         940
MGC                3899
NP                 1291
OGC                8557
OPC                3279
Other              1571
Pv_Ntf3_Trim63     1105
Pv_Tac1            1944
Pv_Vsig2            431
Smc                 586
Sncg                246
Sst_Chrna2_Myh8     538
Sst_Man1a          1355
Sst_Stk33           246
Vip_Chat            500
Vip_Gcnt4           292
Vip_Hcls1           395
Vip_Lipg            245
dtype: int64
(81196, 13)


,cell,sample,barcode,TN,UM,PP,UQ,CM,cluster,tsne1,tsne2,umap-1,umap-2
0,CEMBA171206_3C_AGCGATAGAACCAGGTAAGAGATGTATAGCCT,CEMBA171206_3C,AGCGATAGAACCAGGTAAGAGATGTATAGCCT,138330,128263,128039,64868,0,L6.CT,7.605117,0.697672,-2.125204,-4.396653
1,CEMBA171206_3C_AGCGATAGAACCAGGTAATGACGTCAGGACGT,CEMBA171206_3C,AGCGATAGAACCAGGTAATGACGTCAGGACGT,159133,146663,146348,69167,0,L23.b,-9.492925,-6.200938,8.959074,3.006819
2,CEMBA171206_3C_AGCGATAGAACCAGGTAGGATAACATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTAGGATAACATAGAGGC,19152,17586,17545,8348,0,ASC,14.227358,-11.023748,-6.880754,-2.903271
3,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTAGGCGAAG,14876,13837,13801,6583,0,L6.CT,5.144768,4.790674,-0.955707,-4.635163
4,CEMBA171206_3C_AGCGATAGAACCAGGTATAGCCTTATAGAGGC,CEMBA171206_3C,AGCGATAGAACCAGGTATAGCCTTATAGAGGC,28532,26303,26202,12437,0,L5.IT.b,5.218895,-8.674734,0.796851,4.447933


In [12]:
# meta = pd.read_msgpack(os.path.join(SRC_DIR, 'snmc_and_snatac/study/atac/ATAC.cell_tidy_data.msg'))
# meta_cols = meta.columns.values
# print(meta.shape)
# meta.head()

In [11]:
# meta = meta.rename(columns={'barcode': 'sample', 'sample': 'barcode'})
# meta['cell'] = meta['sample'] + '_' + meta['barcode']
# meta = meta[np.hstack([['cell'], meta_cols])]
# meta.head()

In [7]:
import h5py

In [8]:
reps = ['CEMBA180409_2C', 
        'CEMBA180410_2C',
        'CEMBA171206_3C',
        'CEMBA171207_3C',
        'CEMBA171212_4B',
        'CEMBA171213_4B',
        'CEMBA180104_4B',
        'CEMBA180612_5D',
        # 'CEMBA180605_5D',
        'CEMBA180618_5D',
       ]


genes_all = []
cells_all = []
mat_all = []
for rep in reps:
    print(rep)
    f = h5py.File(os.path.join(SRC_DIR, 'snmc_and_snatac/dataset/atac', rep+'.snap'), 'r')
    # get genes cells mat for one replicate
    genes = f['GM']['name'][:].astype(str)
    cells = np.array([(rep+'_')+barcode for barcode in f['BD']['name'][:].astype(str)])
    idx = f['GM']['idx'][:] - 1
    idy = f['GM']['idy'][:] - 1
    count = f['GM']['count']
    mat = sparse.coo_matrix((count, (idx, idy)), shape=(len(cells), len(genes))).T
    
    # 
    genes_all.append(genes)
    cells_all.append(cells)
    mat_all.append(mat)

CEMBA180409_2C
CEMBA180410_2C
CEMBA171206_3C
CEMBA171207_3C
CEMBA171212_4B
CEMBA171213_4B
CEMBA180104_4B
CEMBA180612_5D
CEMBA180618_5D


In [9]:
for i, genes in enumerate(genes_all):
    if i == 0:
        ref = genes
    else:
        assert np.all(genes == ref)

genes_merge = genes
cells_merge = np.hstack(cells_all)
mat_merge = sparse.hstack(mat_all)

print(genes_merge.shape, cells_merge.shape, mat_merge.shape)

(53278,) (135665,) (53278, 135665)


In [17]:
gene_table = df_genes_v2.reindex(genes_merge).dropna()
assert len(gene_table) == len(genes_merge) #.shape
genes_merge_id = gene_table['gene_id'].values

print(genes.shape)
print(genes_merge_id.shape)
genes_merge_id

(53278,)
(53278,)


array(['ENSMUSG00000037552', 'ENSMUSG00000052609', 'ENSMUSG00000040624',
       ..., 'ENSMUSG00000110345', 'ENSMUSG00000110681',
       'ENSMUSG00000110504'], dtype=object)

In [18]:
gc_mat = GC_matrix(genes_merge_id, cells_merge, mat_merge)


In [19]:
snmcseq_utils.save_gc_matrix(gc_mat, 
                             fout_data_gene, fout_data_cell, fout_data)

In [18]:
meta_new.to_csv(fout_meta, sep="\t", header=True, index=False)